conda env remove -n deepseek-ai

conda create -n deepseek-ai -c conda-forge -y  
conda activate deepseek-ai   
conda install python=3.10 numpy=1.26.4 pillow ipykernel jupyter nb_conda_kernels ipywidgets -c conda-forge -y  
conda install pytorch=2.0.1 -c conda-forge -y
pip install git+https://github.com/huggingface/transformers 
pip install xformers==0.0.21

pip3 install torch==2.0.1+cu118 torchaudio==2.0.2+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118  

git clone https://github.com/deepseek-ai/DeepSeek-VL2
cd DeepSeek-VL2
pip install -e .

python -m ipykernel install [--user] --prefix=C:\Users\techexpert\.conda\envs\deepseek-ai --name deepseek-ai  

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! 🎉")
    device = torch.device("cuda")  # Set device to CUDA
else:
    print("CUDA is NOT available. 🙁")
    device = torch.device("cpu")  # Set device to CPU (or raise an exception)

from transformers import AutoModelForCausalLM
from PIL import Image

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images

specify the path to the model 
 
Alternates:
model_path = "deepseek-ai/deepseek-vl2"  
model_path = "deepseek-ai/deepseek-vl2-small"  

In [ ]:
model_path = "deepseek-ai/deepseek-vl2-tiny"

In [ ]:
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)

In [ ]:
tokenizer = vl_chat_processor.tokenizer

downloads the model to cache folder: C:\Users\techexpert\.cache\huggingface\hub  

In [ ]:
vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
vl_gpt = vl_gpt.to(device) # Move to GPU first
vl_gpt = vl_gpt.to(torch.bfloat16)  # Then change precision
vl_gpt.eval() # Set to eval mode

In [ ]:
# image = Image.open('..\images\dl1.jpg').convert('RGB')
# prompt = "Extract all text found on the image."

image = Image.open('..\images\WalmartReceipt.png').convert('RGB')
prompt = "Extract all text found on the image."

# image = Image.open('..\images\animals.jpg').convert('RGB')
# "Describe the image."

single image conversation example  

In [ ]:
conversation = [
    {
        "role": "<|User|>",
        "content": f"<image>\n|ref|>{prompt}<|/ref|>.",
        "images": [image],
    },
    {"role": "<|Assistant|>", "content": ""},
]

load images and prepare for inputs  
also move the model to GPU  

In [ ]:
model_inputs = vl_chat_processor(
    conversations=conversation,
    images=[image],
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

run image encoder to get the image embeddings  

In [ ]:
inputs_embeds = vl_gpt.prepare_inputs_embeds(**model_inputs)

run the model to get the response  
Important for inference:   
Don't calculate gradients as the model parameters are not updated during inference,   
and this will save time and memory   

In [ ]:
with torch.no_grad():
    outputs = vl_gpt.language.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=model_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2048,
        do_sample=False,
        use_cache=True
    )

In [ ]:
response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)

In [ ]:
print(f"{model_inputs['sft_format'][0]}", response)